In [2]:
import pyforest
data_path = '/Users/jayceepang/msse/ATOM_CODE/original_datasets/' 

In [6]:
def check_labels(datapath): 
  """a function to check that the labels are indeed what we want (a % binding or % inhibition activiy is 
  >= 50% for that molecule to be active against that NEK)"""
  assay_pct_dfs = [file for file in os.listdir(datapath) if file.startswith('NEK')]
  for file in assay_pct_dfs:
    full_file = os.path.join(datapath, file)
    df = pd.read_csv(full_file)
    
    pct_col = [col for col in df.columns if col.startswith('pct_')] 
    len(pct_col) == 1 
    pct_col = pct_col[0]
    invalid_labels = df[df['active']!= (df[pct_col]>=50).astype(int)]
    if invalid_labels.empty: 
      # print(f'correct label assignment based on {pct_col} column')
      pass
    else: 
      print(f'Error on label assignement.')
      print(invalid_labels)
    # check for duplicates here 
    # duplicates = df[df.duplicated()]
    print(f'{df.shape}')
    # duplicates = df[df.duplicated(subset=['base_rdkit_smiles'])]
    # duplicates = df.duplicated()
    duplicates = df[df.duplicated(subset=['base_rdkit_smiles'])]
    if not duplicates.empty:
        print('Duplicate rows found:')
        print(duplicates)
        df_cleaned = df.drop_duplicates()
        print(f'df size original: {df.shape}')
        print(f'remove duplicates. {df_cleaned.shape}')
    duplicates = df[df.duplicated(subset=['compound_id'])]
    if not duplicates.empty:
        print('Duplicate rows found:')
        print(duplicates)
        df_cleaned = df.drop_duplicates()
        print(f'df size original: {df.shape}')
        print(f'remove duplicates. {df_cleaned.shape}')
check_labels(data_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(393, 5)


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

(1404, 5)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(1408, 5)


In [4]:
# must specify filename because the 50pct binding and inhibition files differ
# to-do: discuss how to do this --> this will create a file that is just this but with column specifying split
#       do we want to already integrate a split_uuid like AMPL? 
#       additionally, we still have to use the splits to create the "scaled df" 
def remove_duplicates(datapath, filename):
    df = pd.read_csv(f'{datapath}{filename}')
    duplicates = df.duplicated(subset=['base_rdkit_smiles'])
    if duplicates.any():
        print('Duplicate rows found:')
        print(df[duplicates])
        df_cleaned = df.drop_duplicates(subset=['base_rdkit_smiles'])
        print(f'df size original: {df.shape}')
        print(f'Removed duplicates. New df size: {df_cleaned.shape}')
    else: 
        df_cleaned = df
    return df_cleaned
    
def split_data(datapath, filename, train_ratio=.8, test_ratio=0.2): 
    """filename: NEK#_1_uM_min_50_pct_(binding or inhibition).csv"""
    df = remove_duplicates(datapath, filename)

    # is this extra/too much and should we just assume what labels coincide with majority and minority? 
    # determine majority/minority class 
    #       AMPL takes in 'response column' 
    class_labels = df['active'].value_counts() 
    print(class_labels)
    if len(class_labels)>1: 
        majority_class_label =class_labels.idxmax() 
        majority_num = class_labels.max() 
        minority_class_label = class_labels.idxmin()
        minority_count = class_labels.min()
    df_majority = df[df['active']==majority_class_label]
    df_minority=df[df['active']==minority_class_label]
    # copy to avoid warnings 
    df_majority = df_majority.copy()
    df_minority = df_minority.copy()
    n = round(1/test_ratio) # how else can we do this? i think we should keep kfold splits 
            # 1/.2 = 5 splits 
    kf = KFold(n_splits=n,shuffle=True, random_state=42)
    # majority 
    for i, (_, v_ind) in enumerate(kf.split(df_majority)):
        df_majority.loc[df_majority.index[v_ind], 'fold'] = f"fold{i+1}"
    # minority 
    for i, (_, v_ind) in enumerate(kf.split(df_minority)):
        df_minority.loc[df_minority.index[v_ind], 'fold'] = f"fold{i+1}"
    all_fold_df = pd.concat([df_majority,df_minority])
    
    print(all_fold_df.shape)
    print(all_fold_df.active.value_counts())
    ## actually, this might be perfect jsut to save as "split df" because then 
    # in the next step of creating a dataset, you can choose what fold you want to be the test set 
    return all_fold_df
        
    

# check for duplicates and split and save splits
This demonstrates how it would be done in a 'tuotrial'. 
Either put in your datapath and file name one at a time, or find a way to run a for loop if you're doing a buch of datasets. 

In [8]:
assay_pct_dfs = [file for file in os.listdir(data_path) if file.startswith('NEK')]

def get_bind_inhib(df_name):
    match = re.search(r'_([^_]+)\.csv$',df_name)
    return match.group(1) if match else None

split_dir = '/Users/jayceepang/msse/ATOM_CODE/datasets/'

for i in range(len(assay_pct_dfs)): 
    bind_inhib = get_bind_inhib(assay_pct_dfs[i])
    nek_num = str(assay_pct_dfs[i][3])
    print(assay_pct_dfs[i])
    NEK_name = f'NEK{nek_num}_{bind_inhib}'
    print(NEK_name)
    cleaned_split_df = split_data(data_path, assay_pct_dfs[i])

    # DO WE DO THIS HERE???  
    cleaned_split_df['NEK'] =NEK_name 
    cleaned_split_df.to_csv(f'{split_dir}{NEK_name}_split.csv',index=False)
    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

NEK9_1_uM_min_50_pct_inhibition.csv
NEK9_inhibition


<IPython.core.display.Javascript object>

active
0    351
1     42
Name: count, dtype: int64


...

<IPython.core.display.Javascript object>

active
0    1351
1      57
Name: count, dtype: int64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(1408, 6)
active
0    1351
1      57
Name: count, dtype: int64


# Featurization
should we do this per dataset? (intend to run this in a for loop?)

In [ ]:
def scale_dataset(train, test): 
    # file is the split file (folds)
    # add code to do standard scaler 
    # code to scale then concat 
    scaled_df = pd.concat([train,test])
    return scaled_df

def featurize(datapath, filename, feat_type, test_fold = 1, n_bits=None): 
    # if feat_type == 'MOE': 
        # direct to descriptors files and matchup our split df with the features 
    # if feat_type == 'MFP' or 'ECFP': 
        # create MFP feats? 
        # or should we use ECFP? 
        # specify bits (is not None)  
    # 1. split data based on fold 
    df = pd.read_csv(datapath+filename) # this is the split.csv 
    test = df[df['fold']==test_fold]
    train=df[df['fold']!=test_fold] 
    #### HOW DO WE ACCOUNT FOR VALIDATION? ### 
    # 2. standard scaler 
    scaled_df = scale_dataset(train,test)
    # 3. save scaled_df NO MATTER WHAT.
    # ## MAYBE WE SHOULD SAVE IT IN THE SCALE DF FUNCTION?  ## 
    # 4. perform featurization 
    # 5. save datasets 
    # 6. possibly, call function to prep for training (save trainX.csv, train_y.csv, testX.csv, test_y.csv)
    

In [137]:
# # I'm checking the scaled_descriptors files because I knew there were duplicates in these files 
# desc_dir = '/Users/jayceepang/msse/ATOM_CODE/original_datasets/scaled_descriptors/'
# desc_dfs = [file for file in os.listdir(desc_dir) if file.startswith('NEK')]

# print(desc_dfs)
# for i in range(len(desc_dfs)): 
#     split_data(desc_dir, desc_dfs[i])

In [60]:
1904+140=2044

2044

In [ ]:
id_cols = ['NEK', 'compound_id', 'base_rdkit_smiles','subset', 'active'] 
def over_sampling(data_path,filename, sampling, NEK):
    """Oversample the datasetes using the SMOTE or ADASYN
    Keeps the feature names and id cols
    file_name (full/absolute path): use the scaled dataframe we just created above 'NEK#_(binding/inhibition)_(MOE/MFP)_none_scaled_df.csv'
    sampling (str): 'SMOTE' or 'ADASYN'
    returns: oversampled dataframe
    """
    
    df = pd.read_csv(data_path+filename) # this is the already scaled ver
    feat_cols = list(set(list(df.columns))-set(id_cols))
    
    # train and test 
    train = df[df['subset']=='train'] 
    test =df[df['subset']=='test'] 

    # separate just id cols
    just_ids = ['NEK', 'compound_id', 'base_rdkit_smiles','subset']
    train_just_ids = train[just_ids]
    test_just_ids = test[just_ids]

    # just feats and 'active'
    trainX = train[feat_cols]
    testX = test[feat_cols]
    
    trainy = train['active']
    testy = test['active']
    
    if sampling == 'ADASYN':
        oversample = ADASYN(random_state=42)
    else: 
        oversample = SMOTE(random_state=42)

    
    trainX_temp, trainy_temp = oversample.fit_resample(trainX.to_numpy(), trainy.to_numpy().reshape(-1))
    print(f'train after {sampling}: {trainX_temp.shape}')
    
    trainX_resamp = pd.DataFrame(trainX_temp, columns=feat_cols)
    trainy_resamp = pd.DataFrame(trainy_temp, columns=['active'])

    num_real = len(train)
    num_synthetic = len(trainX_resamp)-num_real
    synthetic_ids = pd.DataFrame({'NEK': [NEK] * num_synthetic,
        'compound_id': [f'synethic_{sampling}_{i}' for i in range(num_synthetic)],
        'base_rdkit_smiles': [f'synethic_{sampling}'] * num_synthetic,
        'subset': ['train']*num_synthetic}) # ,'active':[1]*num_synthetic}

    real_ids = train_just_ids.reset_index(drop=True)
    combined_ids = pd.concat([real_ids,synthetic_ids], ignore_index=True)
    
    train_resamp = pd.concat([combined_ids, trainX_resamp, trainy_resamp[['active']]], axis=1)

    print(train_resamp.columns[train_resamp.columns.duplicated()])
    test_df_final = pd.concat([test_just_ids.reset_index(drop=True),
                               testX.reset_index(drop=True), testy.reset_index(drop=True)],axis=1)

    
    
    # test_df_final['subset'] = 'test'
    # train_resamp = train_resamp.reindex(columns=df.columns)
    # test_df_final = test_df_final.reindex(columns=df.columns)
    
    
    final_df = pd.concat([train_resamp, test_df_final]).reset_index(drop=True)


    return final_df



In [26]:
id_cols = ['NEK', 'compound_id', 'base_rdkit_smiles','subset', 'active'] 
from imblearn.under_sampling import RandomUnderSampler
def undersampler(path,filename,NEK): 
    df = pd.read_csv(path+filename) # this is the already scaled ver
    feat_cols = list(set(list(df.columns))-set(id_cols))
    
    
    # train and test 
    train = df[df['subset']=='train'] 
    test =df[df['subset']=='test'] 

    # separate just id cols
    just_ids = ['NEK', 'compound_id', 'base_rdkit_smiles','subset']
    train_just_ids = train[just_ids]
    test_just_ids = test[just_ids]

    # just feats and 'active'
    trainX = train[feat_cols]
    testX = test[feat_cols]
    
    trainy = train['active']
    testy = test['active']
    
    undersample = RandomUnderSampler(random_state=42)
    
    trainX_temp, trainy_temp = undersample.fit_resample(trainX.to_numpy(), trainy.to_numpy().reshape(-1))
    
    trainX_resamp = pd.DataFrame(trainX_temp, columns=feat_cols)
    trainy_resamp = pd.DataFrame(trainy_temp, columns=['active'])
    
    train_ids_resamp = train_just_ids.iloc[trainX_resamp.index].reset_index(drop=True)
    train_resamp= pd.concat([train_ids_resamp, trainX_resamp,trainy_resamp], axis=1)
    train_resamp['subset'] = 'train'

    test_df_final = pd.concat([test_just_ids.reset_index(drop=True),testX.reset_index(drop=True),testy.reset_index(drop=True)],axis=1)
    test_df_final['subset'] = 'test'
    final_df = pd.concat([train_resamp,test_df_final]).reset_index(drop=True)
    
    return final_df[list(df.columns)]
    

In [29]:
# over_sampling(featurized_filepath, 'NEK2_binding_MFP_none_scaled.csv', "SMOTE", "NEK2_binding")
testing_jaycee =undersampler('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/', 'NEK2_binding_MFP_none.csv',"NEK2_binding")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
testing_jaycee

,NEK,compound_id,base_rdkit_smiles,subset,active,0,1,2,3,4,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,NEK2_binding,kdb_10,CCS(=O)(=O)N1CC(CC#N)(n2cc(-c3ncnc4[nH]ccc34)c...,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NEK2_binding,kdb_100,C[C@@H](Oc1cc(C(=O)Nc2ccc(C(=O)N3CCN(C)CC3)cc2...,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NEK2_binding,kdb_1003,CC1(O)CC(c2nc(-c3ccc4ccc(-c5ccccc5)nc4c3)c3c(N...,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NEK2_binding,kdb_101,COCC(=O)NC/C=C/c1ccc2ncnc(Nc3ccc(Oc4ccc(C)nc4)...,train,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NEK2_binding,kdb_102,COC(=O)c1ccc2c(c1)NC(=O)/C2=C(\Nc1ccc(N(C)C(=O...,train,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,NEK2_binding,kdb_2667,C[C@@H](Oc1cc(-n2cnc3cc(-c4ccnc(NCCS(C)(=O)=O)...,test,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
369,NEK2_binding,kdb_2696,COc1cccc(C2=C(Nc3cc(Cl)c(O)c(Cl)c3)C(=O)NC2=O)c1,test,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
370,NEK2_binding,kdb_2840,COc1cc(Nc2nccc(-c3c(-c4cccc(NC(=O)c5c(F)cccc5F...,test,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
371,NEK2_binding,kdb_2863,CCN(CC)CCOc1ccc(Nc2ncc3cc(-c4c(Cl)cccc4Cl)c(=O...,test,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
id_cols = ['NEK', 'compound_id', 'base_rdkit_smiles','subset', 'active'] ### MAKE SUREFOLD IS NOT IN THIS 
def create_arrays(filepath, root,save_path, printout=True): 
    df = pd.read_csv(file_path+df_filename)
    train = df[df['subset']=='train']
    test=df[df['subset']=='test']
    feat_cols = list(set(list(df.columns))-set(id_cols))
    
    train_y = train['active'].to_numpy().reshape(-1)
    test_y=test['active'].to_numpy().reshape(-1)

    
    trainX = train.drop(columns=id_cols).to_numpy()
    testX = test.drop(columns=id_cols).to_numpy()
    if printout: 
        print(f'train X shape: {trainX.shape}, y: {train_y.shape}, test X: {testX.shape}, y:{test_y.shape}')
    if save_path is not None: 
        trainX = pd.DataFrame(trainX)
        trainX.to_csv(file_path+root+'_trainX.csv', index=False)
   
        trainy_df = pd.DataFrame(train_y)
        trainy_df.to_csv(file_path+root+'_train_y.csv', index=False) 

        testX = pd.DataFrame(testX)
        testX.to_csv(file_path+root+'_testX.csv', index=False)

        testy_df = pd.DataFrame(test_y)
        testy_df.to_csv(file_path+root+'_test_y.csv', index=False) 
        
    return trainX, train_y, testX, test_y

def get_arrays(file_path, df_filename,printout=False):
   # MAKE SURE FOLD IS NOT IN THIS ###### 
    df = pd.read_csv(file_path+df_filename)
    feat_cols = list(set(list(df.columns))-set(id_cols))
    train_df= df[df['subset']=='train']
    test_df = df[df['subset']=='test']
    train_y = train_df['active'].to_numpy().reshape(-1)
    test_y=test_df['active'].to_numpy().reshape(-1)
    train_x_df = train_df.drop(columns='active')

  
    test_x_df = test_df.drop(columns='active')
    
    train_x_df = train_df[feat_cols]
    test_x_df = test_df[feat_cols]

    if printout: 
        print(f'train X shape: {trainX.shape}, y: {train_y.shape}, test X: {testX.shape}, y:{test_y.shape}')

    ### OR read in the data 
    trainX = pd.read_csv(need to put in file name_trainX.csv')
    ... 
        
    return trainX, train_y, testX, test_y

In [ ]:

for nek in neks: 
    for feat in feats: 
        for samp in ['UNDER', 'SMOTE', 'ADASYN']:
            og_df_name = f'{nek}_{feat}_none_scaled.csv'
            if samp == 'UNDER': 
                sampled_df = undersampler(filepath,filename) 
            else: 
                sampled_df=oversampler(filepath,filename, samp, nek) 
            sampled_df.to_csv(f'{save_dir OR same filepath}{nek}_{feat}_{samp}.csv',index=False)
            _, _,_,_=create_arrays(filepath, og_df_name, save_dir OR same filepath) 
            
            

In [ ]:
samplings =['none_scaled','UNDER','SMOTE','ADASYN'] 
for nek in neks: 
    for feat in feats: 
        for samp in samplings:
            root_name = f'{nek}_{feat}_{samp}'
            df = pd.read_csv(fileapth+root_name+'.csv')
            trainX, train_y, testX, test_y=get_arrays(filepath,root_name+'.csv')
            for rf in RF_types: 
                
                model = rf_models(trainX, trainy, testX, testy, rf, {})  # make sure dict and doesn't go to default RF version
                model_name = f'{nek}_{feat}_{samp}_{rf}'
                print(model_name) 
                with open(f'{model_pickle_dir}{model_name}.pkl', 'wb') as f: 
                    pickle.dump(model, f) 
                # add cm and other metrics to this function 
                train_df = gather_rf_results(model, trainX, trainy)
                test_df = gather_rf_results(model, testX, testy)
                train_df['subset'] = 'train' 
                test_df['subset'] = 'test' 
                
                for this_df in [train_df,test_df]: 
                    this_df['model'] = model_name
                    this_df= add_cm(this_df)
                    this_df['NEK'] =nek
                    this_df['feat_type'] = feat
                    this_df['strategy'] = samp
                    this_df['RF_type'] = rf
                train_df.to_csv(f'{results_dir}{model_name}_train.csv',index=False) 
                test_df.to_csv(f'{results_dir}{model_name}_test.csv',index=False) 
                    
                        
                    

# k fold validation

In [ ]:
k=5 
# import numpy as np
from sklearn.model_selection import StratifiedKFold
# X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
# y = np.array([0, 0, 1, 1])
# skf = StratifiedKFold(n_splits=2)
# skf.get_n_splits(X, y)
# print(skf)
# for i, (train_index, test_index) in enumerate(skf.split(X, y)):
#     print(f"Fold {i}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")
# skf = StratifiedKFold(n_splits=k)
# skf.get_n_splits(X, y)
for nek in neks: 
    for feat in feats: 
        for samp in samplings:
            filename = f'{nek}_{feat}_{samp}'
            # call create folds 
            folded_df = create_folds(datapath, filename, k)
            # fold_dataframes = []
                for fold in folded_df['fold'].unique():
                    train_df = folded_df [folded_df ['fold'] != fold].copy()
                    val_df = folded_df [folded_df ['fold'] == fold].copy()
                    
                    train_df['subset'] = 'train'
                    val_df['subset'] = 'validation' # or should we jsut say 'test' 
            
                    fold_df = pd.concat([train_df, val_df])
                    # fold_dataframes.append(fold_df)
                    fold_df.to_csv(f'{FOLDdatapath}{filename}_{fold}.csv', index=False)
            
            
                # combined_folds_df = pd.concat(fold_dataframes, ignore_index=True)